In [1]:
import os
import pandas as pd
from tqdm import trange

In [2]:
year = 2016

In [3]:
data_source = os.path.expanduser('~/data/entso-e/raw')

## Generation

In [4]:
flow_sum = {}
flow_count = {}

for month in trange(1, 13):
    flow_df = pd.read_csv('%s/%d_%02d_PhysicalFlows_12.1.G.csv' % (data_source, year, month), sep='\t')
    flow_df = flow_df[(flow_df['OutAreaTypeCode'] == 'CTY') & (flow_df['InAreaTypeCode'] == 'CTY')]
    
    for i, row in flow_df.iterrows():
        country_in = row.InMapCode
        country_out = row.OutMapCode
        if (country_out, country_in) not in flow_sum.keys():
            flow_sum[(country_out, country_in)] = row.FlowValue
            flow_count[(country_out, country_in)] = 1
        else:
            flow_sum[(country_out, country_in)] += row.FlowValue
            flow_count[(country_out, country_in)] += 1

100%|███████████████████████████████████████████| 12/12 [00:36<00:00,  3.03s/it]


In [5]:
set(flow_count.values())

{2,
 8017,
 8189,
 8192,
 8236,
 8238,
 8239,
 8244,
 8247,
 8678,
 8679,
 8681,
 8683,
 8707,
 8710,
 8723,
 8726,
 8729,
 8731,
 8736,
 8753,
 8758,
 8759,
 8760,
 8774,
 8775,
 8776,
 8778,
 8779,
 8780,
 8781,
 8782,
 8783,
 8784}

In [6]:
asymetric_flow = {country_pairs: flow_sum[country_pairs] / count for country_pairs, count in flow_count.items()
    if count >= 1000}

In [7]:
country_pairs = set(asymetric_flow.keys())

Check that the flows always go both ways:

In [16]:
reversed_country_pairs = {(country_in, country_out) for country_out, country_in in country_pairs}
country_pairs - reversed_country_pairs, reversed_country_pairs - country_pairs

({('BG', 'MK')}, {('MK', 'BG')})

In [19]:
for pair in reversed_country_pairs - country_pairs:
    asymetric_flow[pair] = 0.0

In [18]:
('CH', 'DE') in country_pairs, ('DE', 'CH') in country_pairs

(True, True)

In [20]:
flow = {(country_out, country_in): value - asymetric_flow[country_in, country_out]
        for (country_out, country_in), value in asymetric_flow.items()}

Example: Switzerland is importing from Germany, France, and Austria, but exporting to Italy

In [21]:
flow['CH', 'AT'], flow['CH', 'DE'], flow['CH', 'FR'], flow['CH', 'IT']

(-739.6757479508175,
 -1659.8253563296923,
 -338.3109437613841,
 2224.6450364298726)

In [22]:
out_flow = {country_out: 0. for (country_out, country_in) in country_pairs}
for (country_out, country_in), value in flow.items():
    out_flow[country_out] += value

In [23]:
out_flow['CH']

-513.1670116120215

In [24]:
countries = set(out_flow.keys())
len(countries)

36

In [25]:
sum(out_flow.values())

-1.4921397450962104e-12

## Restriction to PanTaGruEl countries

In [26]:
PanTaGruEl_countries = {"BG", "BE", "IT", "HU", "FR", "AT", "DE", "PL", "CZ", "NL",
                        "GR", "CH", "RO", "ES", "SI", "PT", "LU", "HR", "AL", "DK",
                        "RS", "SK", "BA", "ME", "MK"}

In [27]:
PanTaGruEl_countries - countries

{'LU'}

In [19]:
# PanTaGruEl_out_flow = {country: out_flow[country] for country in PanTaGruEl_countries}

In [20]:
# sum(PanTaGruEl_out_flow.values())

In [28]:
PanTaGruEl_flow = {(country_out, country_in): value for (country_out, country_in), value in flow.items()
                   if country_in in PanTaGruEl_countries and country_out in PanTaGruEl_countries}

In [29]:
PanTaGruEl_out_flow = {country_out: 0. for (country_out, country_in) in PanTaGruEl_flow.keys()}
for (country_out, country_in), value in PanTaGruEl_flow.items():
    PanTaGruEl_out_flow[country_out] += value

In [30]:
for country in PanTaGruEl_countries - countries:
    PanTaGruEl_out_flow[country] = 0.0

In [31]:
sum(PanTaGruEl_out_flow.values())

-9.094947017729282e-13

In [32]:
PanTaGruEl_out_flow

{'DE': 5663.423467602068,
 'SI': 127.70901748005349,
 'HR': -717.6638546806478,
 'BE': -725.6580737445695,
 'ES': -1454.9508520247125,
 'FR': 3016.90801600579,
 'RS': 167.4573649872896,
 'DK': -282.05742707383797,
 'BA': 425.84037039245504,
 'PL': 234.59811361566136,
 'RO': 664.2823519529622,
 'NL': -991.2699875031536,
 'GR': -838.1982979681414,
 'BG': 189.87581208274827,
 'CH': -513.1670116120215,
 'AT': -993.9815482695793,
 'CZ': 1251.1321745844712,
 'SK': -537.4509191708855,
 'HU': -963.7807658364916,
 'PT': 573.056867030968,
 'IT': -4133.250492927997,
 'AL': 4.8887937402344335,
 'MK': -136.80375277013076,
 'ME': -30.939365892533118,
 'LU': 0.0}

In [33]:
PanTaGruEl_out_flow_df = pd.DataFrame(PanTaGruEl_out_flow.items(), columns=['country', 'out_flow'])

In [34]:
PanTaGruEl_out_flow_df.to_csv('pantagruel_out_flow.csv', index=False)